In [64]:
!pip install praw


In [65]:
!pip install vaderSentiment
!pip install scikit-learn
!pip install pandas

In [66]:
import praw
import pandas as pd
import re

# Set up Reddit API credentials
reddit = praw.Reddit(
    client_id='1CCSWzp4KE6Cudb1WlZ-QQ',  # Your client ID
    client_secret='km8s1OpAWlorSIRCxdKCLAVx1o0QQg',  # Your client secret
    user_agent='Stock sentiment analysis'  # Your user agent
)

# Choose a subreddit (you can change this to any subreddit you want)
subreddit = reddit.subreddit('stocks')

# Scrape the latest 10 posts
posts = []
for submission in subreddit.new(limit=10):
    posts.append({
        'title': submission.title,
        'score': submission.score,
        'url': submission.url,
        'created': submission.created_utc
    })

# Store data in a DataFrame
df = pd.DataFrame(posts)

# Function to clean text
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters, numbers, and extra spaces
    text = re.sub(r'[^A-Za-z\s]', '', text)
    text = text.lower()
    return text

# Apply the cleaning function to the titles
df['cleaned_title'] = df['title'].apply(clean_text)

# Show the cleaned DataFrame
df.head()


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



,title,score,url,created,cleaned_title
0,Nordstrom’s chances of going private?,6,https://www.reddit.com/r/stocks/comments/1h9fw...,1.733653e+09,nordstroms chances of going private
1,Best strategy for investing in SPY:,19,https://www.reddit.com/r/stocks/comments/1h998...,1.733627e+09,best strategy for investing in spy
2,Help understanding JBT merger,3,https://www.reddit.com/r/stocks/comments/1h97v...,1.733622e+09,help understanding jbt merger
3,SMCI Recovery and loose DD,18,https://www.reddit.com/r/stocks/comments/1h96w...,1.733619e+09,smci recovery and loose dd
4,Is it true most people lost money when they st...,0,https://www.reddit.com/r/stocks/comments/1h95r...,1.733616e+09,is it true most people lost money when they st...


In [67]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to get sentiment score
def get_sentiment(title):
    sentiment_score = analyzer.polarity_scores(title)
    return sentiment_score['compound']  # compound score represents overall sentiment

# Apply the sentiment analysis function to the cleaned titles
df['sentiment'] = df['cleaned_title'].apply(get_sentiment)

# Show the dataframe with sentiment scores
df.head()


,title,score,url,created,cleaned_title,sentiment
0,Nordstrom’s chances of going private?,6,https://www.reddit.com/r/stocks/comments/1h9fw...,1.733653e+09,nordstroms chances of going private,0.2023
1,Best strategy for investing in SPY:,19,https://www.reddit.com/r/stocks/comments/1h998...,1.733627e+09,best strategy for investing in spy,0.6369
2,Help understanding JBT merger,3,https://www.reddit.com/r/stocks/comments/1h97v...,1.733622e+09,help understanding jbt merger,0.4019
3,SMCI Recovery and loose DD,18,https://www.reddit.com/r/stocks/comments/1h96w...,1.733619e+09,smci recovery and loose dd,-0.3182
4,Is it true most people lost money when they st...,0,https://www.reddit.com/r/stocks/comments/1h95r...,1.733616e+09,is it true most people lost money when they st...,0.0571


In [68]:
# Add a column with time since post creation (in days)
df['time_since_posted'] = (pd.to_datetime('now') - pd.to_datetime(df['created'], unit='s')).dt.days

# Show the dataframe with all the features
df.head()


,title,score,url,created,cleaned_title,sentiment,time_since_posted
0,Nordstrom’s chances of going private?,6,https://www.reddit.com/r/stocks/comments/1h9fw...,1.733653e+09,nordstroms chances of going private,0.2023,0
1,Best strategy for investing in SPY:,19,https://www.reddit.com/r/stocks/comments/1h998...,1.733627e+09,best strategy for investing in spy,0.6369,0
2,Help understanding JBT merger,3,https://www.reddit.com/r/stocks/comments/1h97v...,1.733622e+09,help understanding jbt merger,0.4019,0
3,SMCI Recovery and loose DD,18,https://www.reddit.com/r/stocks/comments/1h96w...,1.733619e+09,smci recovery and loose dd,-0.3182,0
4,Is it true most people lost money when they st...,0,https://www.reddit.com/r/stocks/comments/1h95r...,1.733616e+09,is it true most people lost money when they st...,0.0571,0


In [69]:
# Creating the target variable (1 if sentiment > 0, else 0)
df['target'] = (df['sentiment'] > 0).astype(int)

# Features: sentiment score, post score, and time since posted
X = df[['sentiment', 'score', 'time_since_posted']]

# Target: Positive sentiment
y = df['target']


In [70]:
df['target'].value_counts()


,count
target,
1,5
0,5


In [71]:
# Add a feature for the length of the post title
df['title_length'] = df['cleaned_title'].apply(len)

# Add a feature for the number of comments on the post
df['num_comments'] = df['score']  # Placeholder, you might need actual comments data

# Update your feature set to include the new features
X = df[['sentiment', 'score', 'time_since_posted', 'title_length', 'num_comments']]


In [72]:
from sklearn.ensemble import RandomForestClassifier

# Train a Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_resampled, y_train_resampled)

# Make predictions
y_pred_rf = rf_model.predict(X_test_resampled)

# Evaluate the model
accuracy_rf = accuracy_score(y_test_resampled, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test_resampled, y_pred_rf)

print(f"Random Forest Accuracy: {accuracy_rf}")
print(f"Random Forest Confusion Matrix: \n{conf_matrix_rf}")


Random Forest Accuracy: 0.5
Random Forest Confusion Matrix: 
[[0 1]
 [0 1]]


In [73]:
!pip install xgboost


In [74]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

# Train an XGBoost model
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set
y_pred_xgb = xgb_model.predict(X_test_resampled)

# Evaluate the model
accuracy_xgb = accuracy_score(y_test_resampled, y_pred_xgb)
precision_xgb = precision_score(y_test_resampled, y_pred_xgb)
recall_xgb = recall_score(y_test_resampled, y_pred_xgb)
f1_xgb = f1_score(y_test_resampled, y_pred_xgb)
conf_matrix_xgb = confusion_matrix(y_test_resampled, y_pred_xgb)

# Print the evaluation metrics
print(f"XGBoost Accuracy: {accuracy_xgb}")
print(f"XGBoost Precision: {precision_xgb}")
print(f"XGBoost Recall: {recall_xgb}")
print(f"XGBoost F1-Score: {f1_xgb}")
print(f"XGBoost Confusion Matrix: \n{conf_matrix_xgb}")


XGBoost Accuracy: 1.0
XGBoost Precision: 1.0
XGBoost Recall: 1.0
XGBoost F1-Score: 1.0
XGBoost Confusion Matrix: 
[[1 0]
 [0 1]]


In [75]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(XGBClassifier(random_state=42), X, y, cv=5, scoring='accuracy')

print(f"Cross-validation scores: {cv_scores}")
print(f"Average CV accuracy: {cv_scores.mean()}")


Cross-validation scores: [1.  1.  1.  0.5 1. ]
Average CV accuracy: 0.9


In [76]:
!pip install asyncpraw


In [77]:
import asyncpraw
import pandas as pd

# Create an instance of the async Reddit API
reddit = asyncpraw.Reddit(
    client_id='1CCSWzp4KE6Cudb1WlZ-QQ',  # Your client ID
    client_secret='km8s1OpAWlorSIRCxdKCLAVx1o0QQg',  # Your client secret
    user_agent='Stock sentiment analysis'  # Your user agent
)

# Async function to scrape Reddit data
async def scrape_reddit():
    new_posts = []

    # Use async for and await to fetch posts from Reddit
    subreddit = await reddit.subreddit('stocks')  # Await subreddit to get the correct object
    async for submission in subreddit.new(limit=5):  # Fetch the latest 5 posts
        new_posts.append({
            'title': submission.title,
            'score': submission.score,
            'url': submission.url,
            'created': submission.created_utc
        })

    return new_posts

# Use await to get the posts
posts = await scrape_reddit()

# Convert the new data into a DataFrame
new_df = pd.DataFrame(posts)

# Display the results
new_df.head()


,title,score,url,created
0,Nordstrom’s chances of going private?,4,https://www.reddit.com/r/stocks/comments/1h9fw...,1.733653e+09
1,Best strategy for investing in SPY:,23,https://www.reddit.com/r/stocks/comments/1h998...,1.733627e+09
2,Help understanding JBT merger,2,https://www.reddit.com/r/stocks/comments/1h97v...,1.733622e+09
3,SMCI Recovery and loose DD,19,https://www.reddit.com/r/stocks/comments/1h96w...,1.733619e+09
4,Is it true most people lost money when they st...,0,https://www.reddit.com/r/stocks/comments/1h95r...,1.733616e+09


In [78]:
# Clean the new data
new_df['cleaned_title'] = new_df['title'].apply(clean_text)

# Extract sentiment scores
new_df['sentiment'] = new_df['cleaned_title'].apply(get_sentiment)

# Add time since post creation (in days)
import pandas as pd
new_df['time_since_posted'] = (pd.to_datetime('now') - pd.to_datetime(new_df['created'], unit='s')).dt.days

# Prepare the feature set
X_new = new_df[['sentiment', 'score', 'time_since_posted']]


In [79]:
# Make predictions on the new data
new_df['predicted_sentiment'] = xgb_model.predict(X_new)

# Display the results
print(new_df[['title', 'predicted_sentiment']])


                                               title  predicted_sentiment
0             Nordstrom’s chances of going private?                     1
1                Best strategy for investing in SPY:                    1
2                      Help understanding JBT merger                    1
3                        SMCI Recovery and loose DD                     0
4  Is it true most people lost money when they st...                    1


In [82]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

# Replace with actual labels for new data
new_df['actual_sentiment'] = [1, 1, 1, 0, 1]  # Replace with actual labels

# Evaluate the model
accuracy_new = accuracy_score(new_df['actual_sentiment'], new_df['predicted_sentiment'])
precision_new = precision_score(new_df['actual_sentiment'], new_df['predicted_sentiment'])
recall_new = recall_score(new_df['actual_sentiment'], new_df['predicted_sentiment'])
f1_new = f1_score(new_df['actual_sentiment'], new_df['predicted_sentiment'])
conf_matrix_new = confusion_matrix(new_df['actual_sentiment'], new_df['predicted_sentiment'])

# Print the evaluation metrics
print(f"Accuracy on New Data: {accuracy_new}")
print(f"Precision on New Data: {precision_new}")
print(f"Recall on New Data: {recall_new}")
print(f"F1-Score on New Data: {f1_new}")
print(f"Confusion Matrix on New Data: \n{conf_matrix_new}")


Accuracy on New Data: 1.0
Precision on New Data: 1.0
Recall on New Data: 1.0
F1-Score on New Data: 1.0
Confusion Matrix on New Data: 
[[1 0]
 [0 4]]
